Running inference on image pairs
============================

If you want to use a pre-trained PWC-Net model on your own set of images, you can pass a list of image pairs to a `ModelPWCNet` object using its  `predict_from_img_pairs()` method, as demonstrated here.

Below, look for `TODO` references and customize this notebook based on your own needs.

In [1]:
"""
pwcnet_predict_from_img_pairs.ipynb

Run inference on a list of images pairs.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)
"""
from __future__ import absolute_import, division, print_function
from copy import deepcopy
# from skimage.io import imread
from cv2 import imread
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TEST_OPTIONS
from visualize import zmf_save_flows_c
import rawpy
import numpy as np
import glob,os

In [2]:
# TODO: Set device to use for inference
# Here, we're using a GPU (use '/device:CPU:0' to run inference on the CPU)
gpu_devices = ['/device:GPU:0']  
controller = '/device:GPU:0'

# TODO: Set the path to the trained model (make sure you've downloaded it first from http://bit.ly/tfoptflow)
# ckpt_path = '../../../data/pretrained_model/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'

# ckpt_path = '../../../workplace/3-noise-model/final/zmf-origChair/pwcnet.ckpt-50000'
# ckpt_path = '../../../workplace/3-noise-model/final/zmf-try-my-raw-6-pwcnet-lg-6-2-cyclic/pwcnet.ckpt-50000'
# ckpt_path = '../../../workplace/3-noise-model/final/train-on-white-noise/pwcnet.ckpt-50000'
# ckpt_path = '../../../workplace/3-noise-model/zmf-finetune/pwcnet.ckpt-48000'

# ckpt_path = '../../../workplace/3-noise-model/mix/zmf-allmyppm/pwcnet.ckpt-50000'
ckpt_path = '../../../workplace/3-noise-model/mix/zmf-mixwhite/pwcnet.ckpt-50000'

In [3]:
# Build a list of image pairs to process
img_pairs = []
names = []

# PNG
# image_path1 = '../../../workplace/1-prove-bad/eva/eva-base/my-simple/6_scale_img1.png'
# image_path2 = '../../../workplace/1-prove-bad/eva/eva-base/my-simple/6_scale_img2.png'
# image_path1 = '../../../data/dataset/newOptFlow/1/(1).JPG'
# image_path2 = '../../../data/dataset/newOptFlow/2/(1).JPG'
# image1, image2 = imread(image_path1), imread(image_path2)
# img_pairs.append((image1, image2))

# ARW
canon_im1num = [1,1,2,10,10,11,21,21,21,21,22,22,22,24,25,31,31,31,32,32,33,41,43,51,54] # 25
canon_im2num = [2,3,3,11,12,12,22,24,25,26,23,24,27,26,27,32,33,34,33,34,34,42,45,52,55]

# sony_im1num = [1,1,1,2,11,11,12,15] # 8
# sony_im2num = [2,3,4,4,12,13,13,16]
sony_im1num = [1,1] # 8
sony_im2num = [2,3]
fuji_im1num = [1,1,1,2,2,3,5,6,11,11,11,12,12,12,14,14,15] # 17
fuji_im2num = [2,3,4,3,4,4,8,7,12,16,17,13,14,15,16,17,17]

def crop(im,h,w,sample_every_n_pixels):
    h0 = im.shape[0]
    w0 = im.shape[1]
    if (h0 < h or w0 < w):
        print("bad crop")
        return im
    newim = im[0:h:sample_every_n_pixels,0:w:sample_every_n_pixels,:]
    return newim

for i in range(0,2):
    foldernum1 = canon_im1num[i]
    foldernum2 = canon_im2num[i]
    rawnum = len(glob.glob('../../../data/dataset/newOptFlow/canon/%d/*.CR2'%(foldernum1)))
    for j in range(rawnum):
        image_path1 = '../../../data/dataset/newOptFlow/canon/%d/(%d).CR2'%(foldernum1,j+1)
        image_path2 = '../../../data/dataset/newOptFlow/canon/%d/(%d).CR2'%(foldernum2,j+1)
#         image_path1 = '../../../data/dataset/newOptFlow/sony/1/(1).ARW'
#         image_path2 = '../../../data/dataset/newOptFlow/sony/2/(1).ARW'

        raw1 = rawpy.imread(image_path1)
        raw2 = rawpy.imread(image_path2)

        im1 = raw1.raw_image_visible.astype(np.float32)
        im1 = (im1 - 2047) / (16383 - 2047)
        ratio = 0.3 / np.mean(im1)
        im1 = np.minimum(np.maximum(im1*ratio,0.0),1.0)

        im2 = raw2.raw_image_visible.astype(np.float32)
        im2 = (im2 - 2047) / (16383 - 2047)
        ratio = 0.3 / np.mean(im2)
        im2 = np.minimum(np.maximum(im2*ratio,0.0),1.0)
        print(ratio)

        im1 = np.expand_dims(im1, axis=2)
        H = im1.shape[0]
        W = im1.shape[1]
        image1 = np.concatenate((im1[0:H:2, 0:W:2,:], #r
                              (im1[0:H:2, 1:W:2,:]+im1[1:H:2, 0:W:2,:])/2.0, #g
                              im1[1:H:2, 1:W:2,:]), axis=2) #b
        im2 = np.expand_dims(im2, axis=2)
        H = im2.shape[0]
        W = im2.shape[1]
        image2 = np.concatenate((im2[0:H:2, 0:W:2,:], #r
                              (im2[0:H:2, 1:W:2,:]+im2[1:H:2, 0:W:2,:])/2.0, #g
                              im2[1:H:2, 1:W:2,:]), axis=2) #b
        image1 = crop(image1,1920,2944,2)
        image2 = crop(image2,1920,2944,2)
        img_pairs.append((image1, image2))
        n = '%d-%d_%d'%(foldernum1,foldernum2,j+1)
        names.append(n)

5.873712089583721
8.272701420914863
12.545421771236596
19.745510215302943
33.269437679478166
52.38627026105759
82.9513810303007
148.5211844346308
232.21744455597334
5.847769296614696
8.417754474869234
12.768816997407805
19.821441508042135
33.42236491762654
52.9145215472365
83.13670222032496
149.4791437361091
226.40215140787717


In [4]:
# Configure the model for inference, starting with the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TEST_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = ckpt_path
nn_opts['batch_size'] = 1
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# We're running the PWC-Net-large model in quarter-resolution mode
# That is, with a 6 level pyramid, and upsampling of level 2 by 4 in each dimension as the final flow prediction
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# The size of the images in this dataset are not multiples of 64, while the model generates flows padded to multiples
# of 64. Hence, we need to crop the predicted flows to their original size
nn_opts['adapt_info'] = (1, 436, 1024, 2)

In [5]:
# Instantiate the model in inference mode and display the model configuration
nn = ModelPWCNet(mode='test', options=nn_opts)
# nn.print_config()

W1104 21:33:50.031205 139737911805760 deprecation_wrapper.py:119] From /home/zhangmf/Documents/Github/tfoptflow/tfoptflow/model_base.py:59: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1104 21:33:50.112115 139737911805760 deprecation_wrapper.py:119] From /home/zhangmf/Documents/Github/tfoptflow/tfoptflow/model_base.py:77: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1104 21:33:50.114075 139737911805760 deprecation_wrapper.py:119] From /home/zhangmf/Documents/Github/tfoptflow/tfoptflow/model_base.py:82: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1104 21:33:51.715600 139737911805760 deprecation_wrapper.py:119] From /home/zhangmf/Documents/Github/tfoptflow/tfoptflow/model_base.py:86: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W1104 21:33:51.716507 139737911805760 deprecation_wrapper.py:119] From /

Building model...


W1104 21:33:52.503699 139737911805760 deprecation.py:506] From /home/zhangmf/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1104 21:33:52.748478 139737911805760 deprecation.py:323] From /home/zhangmf/Documents/Github/tfoptflow/tfoptflow/model_pwcnet.py:1221: conv2d_transpose (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
W1104 21:33:55.309040 139737911805760 deprecation_wrapper.py:119] From /home/zhangmf/Documents/Github/tfoptflow/tfoptflow/model_pwcnet.py:1590: The name tf.image.resize_bilinear is deprecated. Please use tf.compat.v1.image.resize_bilinear instead.

W1104 21:33:5

... model built.
Loading model checkpoint ../../../workplace/3-noise-model/mix/zmf-mixwhite/pwcnet.ckpt-50000 for eval or testing...

... model loaded


In [6]:
# Generate the predictions and display them
pred_labels = nn.predict_from_img_pairs(img_pairs, batch_size=1, verbose=False)
zmf_save_flows_c(names, img_pairs, pred_labels)